### t-SNE plot generation

In [3]:
import pandas as pd
from sklearn.manifold import TSNE, SpectralEmbedding
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

In [4]:
vgg_df = pd.read_csv('../wandb_results/vgg_cifar10_wandb_export.csv')

In [5]:
features = ['epochs', 'batch_size', 'learning_rate', 'adam_epsilon']
vgg_X = vgg_df[features].values
vgg_y = vgg_df['accuracy_diff'].values

In [ ]:
vgg_X_embedded = SpectralEmbedding(n_components=2).fit_transform(vgg_X)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
img = plt.scatter(vgg_X_embedded[:, 0],vgg_X_embedded[:, 1] , c= vgg_y, cmap="viridis")
fig.colorbar(img, orientation="vertical", label="Accuracy Differnece")
ax.set_xlabel('Spectral Component 1')
ax.set_ylabel('Spectral Component 2')
plt.savefig('vgg_spectral_params.png', bbox_inches="tight", dpi=200)

In [ ]:
total_max = np.max([np.max(vgg_mi), np.max(dense_mi), np.max(resnet_mi)])

In [6]:
from sklearn.feature_selection import mutual_info_regression

In [23]:
vgg_mi = mutual_info_regression(vgg_X, vgg_y)
# vgg_mi /= np.max(vgg_mi)
vgg_mi

array([0.        , 0.12358599, 0.04304035, 0.15327897])

In [8]:
resnet_df = pd.read_csv('../wandb_results/resnet_cifar10_wandb_export.csv')
features = ['epochs', 'batch_size', 'learning_rate', 'adam_epsilon']
resnet_X = resnet_df[features].values
resnet_y = resnet_df['accuracy_diff'].values
resnet_mi = mutual_info_regression(resnet_X, resnet_y)
resnet_mi /= np.max(resnet_mi)
resnet_mi

array([0.68144608, 1.        , 0.25444389, 0.42359379])

In [9]:
dense_df = pd.read_csv('../wandb_results/densenet_cifar10_wandb_export.csv')
features = ['epochs', 'batch_size', 'learning_rate', 'adam_epsilon']
dense_X = dense_df[features].values
dense_y = dense_df['accuracy_diff'].values
dense_mi = mutual_info_regression(dense_X, dense_y)
dense_mi /= np.max(dense_mi)
dense_mi

array([1.        , 0.12302519, 0.24898544, 0.12779857])

In [ ]:
sum(vgg_mi)

In [ ]:
mi_df = pd.DataFrame(columns=['Compared Stat', 'MI', 'Model'])

In [ ]:
mi_df

In [ ]:
features_plot = ["Epochs", "Batch Size", "Learning Rate", "Adam Epsilon"]

In [ ]:
for i in range(4):
    mi_df = mi_df.append({'MI':vgg_mi[i], 'Compared Stat': features_plot[i], 'Model':'VGG'}, ignore_index=True)
    mi_df = mi_df.append({'MI':resnet_mi[i], 'Compared Stat': features_plot[i], 'Model':'ResNet'}, ignore_index=True)
    mi_df = mi_df.append({'MI':dense_mi[i], 'Compared Stat': features_plot[i], 'Model':'DenseNet'}, ignore_index=True)

In [ ]:
mi_df

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.barplot(x="Compared Stat", y="MI", hue="Model", data=mi_df, palette="viridis")
plt.ylabel("Normalized Mutual Information")
fig = plt.gcf()
fig.set_size_inches(11, 8)
plt.tight_layout()
plt.savefig('totalmax_normalized_mutual_info_sklearn.jpeg', dpi=200)

# 2D Mutual Information

In [17]:
import numpy as np

from scipy.special import gamma,psi
from scipy import ndimage
from scipy.linalg import det
from numpy import pi

from sklearn.neighbors import NearestNeighbors

EPS = np.finfo(float).eps

In [18]:
def mutual_information_2d(x, y, sigma=1, normalized=False):
    """
    Computes (normalized) mutual information between two 1D variate from a
    joint histogram.
    Parameters
    ----------
    x : 1D array
        first variable
    y : 1D array
        second variable
    sigma: float
        sigma for Gaussian smoothing of the joint histogram
    Returns
    -------
    nmi: float
        the computed similariy measure
    """
    bins = (256, 256)

    jh = np.histogram2d(x, y, bins=bins)[0]

    # smooth the jh with a gaussian filter of given sigma
    ndimage.gaussian_filter(jh, sigma=sigma, mode='constant',
                                 output=jh)

    # compute marginal histograms
    jh = jh + EPS
    sh = np.sum(jh)
    jh = jh / sh
    s1 = np.sum(jh, axis=0).reshape((-1, jh.shape[0]))
    s2 = np.sum(jh, axis=1).reshape((jh.shape[1], -1))

    # Normalised Mutual Information of:
    # Studholme,  jhill & jhawkes (1998).
    # "A normalized entropy measure of 3-D medical image alignment".
    # in Proc. Medical Imaging 1998, vol. 3338, San Diego, CA, pp. 132-143.
    if normalized:
        mi = ((np.sum(s1 * np.log(s1)) + np.sum(s2 * np.log(s2)))
                / np.sum(jh * np.log(jh))) - 1
    else:
        mi = ( np.sum(jh * np.log(jh)) - np.sum(s1 * np.log(s1))
               - np.sum(s2 * np.log(s2)))

    return mi

In [22]:
mutual_information_2d(vgg_df['adam_epsilon'], np.array(vgg_y))

1.4249600116580297